In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#full_training = pd.read_csv('Datasets/crime_merged1.csv')
#full_training = pd.read_csv('Datasets/test_training_merged.csv')
#full_training = pd.read_csv('Datasets/clean_merged4.csv')
#full_training = pd.read_csv('Datasets/clean_merged5_gunpermit.csv')
#full_training = pd.read_csv('Datasets/merged_trainTest_U_gunpermit.csv')
#full_training = pd.read_csv('Datasets/kenny_1.csv')
full_training = pd.read_csv('Datasets/kenny_training_w_testing.csv')

In [ ]:
full_training = full_training.iloc[:,1:]
full_training.income.describe()
print(full_training.shape)
full_training.head()

In [ ]:
from sklearn.preprocessing import scale
full_training['scaled_shots'] = scale(full_training.ShotsClean)
full_training['scaled_pop'] = scale(full_training.estimate2016)
full_training['scaled_income'] = scale(full_training.income)
full_training['scaled_age'] = scale(full_training.NewAge)
#full_training['imputed_age'] = full_training.NewAge.replace(0, np.mean(full_training.loc[full_training.NewAge!=0,'NewAge']))
#full_training['imputed_age'] = full_training.NewAge.replace(0,15)

In [23]:
#training_noU = full_training
training_noU = full_training.loc[full_training.Fatal != 'U',:]
# features_list = ['Fatal','SubjectRace', 'SubjectGender', 'SubjectArmed',
#                                'scaled_shots', 'AgeGroup', 'NumberOfOfficers',
#                                    'Month', 'Day', 'scaled_pop', 'mainOfficerRace','income']



features_list2 = ['Fatal','SubjectRace', 'SubjectGender', 'SubjectArmed',
                'NewAge', 'NumberOfOfficers', 'ShotsClean', 'mainOfficerRace',
                  'Year','Month','Day','estimate2016', 'income', 
                  'X2010', 'X2011', 'X2012','X2013','X2014', 'X2015','X2016']



training_noU_feature = training_noU.loc[:, features_list2].dropna(axis=0)

X = training_noU_feature.drop('Fatal', axis=1)

X_dummy = pd.get_dummies(X)
y = training_noU_feature.Fatal
print(X_dummy.shape)
print(y.shape)


(3421, 33)
(3421,)


## Splitting into testing and training

In [24]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X_dummy, y, test_size=0.2, random_state=1234567)

print('train_features dimensions:', train_X.shape)
print('test_features dimensions:', test_X.shape)
print('train_labels dimensions:', train_y.shape)
print('test_labels dimensions:', test_y.shape)
print('The goal is to beat the classification rate of:', y.value_counts(normalize=True)[0])
print(train_y.value_counts(normalize=True))
print(test_y.value_counts(normalize=True))


train_features dimensions: (2736, 33)
test_features dimensions: (685, 33)
train_labels dimensions: (2736,)
test_labels dimensions: (685,)
The goal is to beat the classification rate of: 0.6539023677287343
N    0.658991
F    0.341009
Name: Fatal, dtype: float64
N    0.633577
F    0.366423
Name: Fatal, dtype: float64


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV


# xgboost causing warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
models = []
#models.append(('LogisticRegression', LogisticRegression()))
#models.append(("SVC", SVC(C=100, gamma=0.01)))
#models.append(("KNeighborsClassifier", KNeighborsClassifier(n_neighbors=10)))
models.append(("RandomForestClassifier_gini", RandomForestClassifier(n_estimators=500,
                                                                criterion='gini',
                                                                max_depth=8,
                                                                    max_features=5)))
#models.append(("RandomForestClassifier_entropy", RandomForestClassifier(n_estimators=500,
#                                                                criterion = 'entropy',
#                                                                max_depth=8,
#                                                                       max_features=5)))
models.append(('XGBoost', XGBClassifier(learning_rate=0.01, n_estimators=300,
                                       max_depth= 3)))
#models.append(('Adaboost', AdaBoostClassifier(learning_rate=0.1, n_estimators=100)))
#models.append(('MLP', MLPClassifier(solver='lbfgs', alpha=1e-5, 
                                     #hidden_layer_sizes=(3, 2), random_state=1)))

results = []
names = []
for name,model in models:
    result = cross_val_score(model, train_X, train_y, cv=5)
    names.append(name)
    results.append(result)

mean_results = []
sd_results = []
for model in results:
    mean_results.append(model.mean())
    sd_results.append(model.std())

results_df = pd.DataFrame({'Model': names,
              'Mean Classification Rate': mean_results,
              'Standard Deviation': sd_results
             })
results_df = results_df.iloc[:,[1,0,2]]

In [ ]:
results_df.sort_values(by = 'Mean Classification Rate', ascending=False)

## XGBoost

In [ ]:
xgb = XGBClassifier(random_state=12345)
#xgb.fit(train_features,train_labels, verbose=True)

param_grid_xgb = {
    'learning_rate': [0.01,0.1],
    'n_estimators': list(range(1,102,5)),
    'max_depth': list(range(2,10))
}
CV_xgb = GridSearchCV(n_jobs=-1, estimator=xgb, param_grid = param_grid_xgb, cv=5,
                     verbose=2, return_train_score=True)
CV_xgb.fit(train_X,train_y)

In [ ]:
mean_test = pd.DataFrame({'Accuracy': CV_xgb.cv_results_['mean_test_score']})
params = pd.DataFrame(CV_xgb.cv_results_['params'])
accuracy_result = params.join(mean_test).sort_values('Accuracy', ascending=False)
accuracy_result.head()

In [25]:
from sklearn.metrics import accuracy_score

xgb = XGBClassifier(learning_rate=0.01, n_estimators=500, max_depth=4, subsample = 0.8)
xgb.fit(train_X, train_y)
xgb_pred_class = xgb.predict(test_X)

print('Classification Rate:', accuracy_score(test_y, xgb_pred_class))
pd.crosstab(xgb_pred_class, test_y, margins=False)

Classification Rate: 0.708029197080292


Fatal,F,N
row_0,,
F,89,38
N,162,396


## Random Forest

In [26]:
rF = RandomForestClassifier(n_estimators=1000, criterion = 'gini', max_depth=10,
                            max_features = 8, oob_score=True)
rF.fit(train_X, train_y)
print('OOB score:', rF.oob_score_)
rF_pred_class = rF.predict(test_X)

print('Classification Rate:', accuracy_score(rF_pred_class, test_y))
print(pd.crosstab(rF_pred_class,test_y))
#pd.DataFrame({'Feature': train_X.columns, 'Importances':rF.feature_importances_}).sort_values(by='Importances', ascending=False)

OOB score: 0.7134502923976608
Classification Rate: 0.7094890510948905
Fatal    F    N
row_0          
F       87   35
N      164  399


## Stacked Ensemble with mlextend

In [27]:
from sklearn import model_selection
from mlxtend.classifier import StackingClassifier

clf1 =  XGBClassifier(learning_rate=0.01, n_estimators=461, max_depth=4, subsample = 0.8)
clf2 =  XGBClassifier(learning_rate=0.1, n_estimators=81, max_depth=3, subsample = 0.8)
clf3 = RandomForestClassifier(n_estimators=800, criterion = 'gini', max_depth=10, max_features = 4)
#clf4 = RandomForestClassifier(n_estimators=800, criterion = 'entropy', max_depth=10, max_features = 4)

#clf3 = KNeighborsClassifier(n_neighbors=10)
#clf3= AdaBoostClassifier(learning_rate=0.1, n_estimators=500)

lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          meta_classifier=lr)

train_y_numeric = train_y.replace({'F':1, 'N':0})

In [28]:
print('5-fold cross validation:\n')

for clf, label in zip([clf1, clf2, clf3, sclf], 
                      ['XGB', 
                       'XGB2',
                       'RF gini',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, train_X, train_y_numeric, 
                                              cv=5, scoring='accuracy')
    print("Accuracy: %0.10f (+/- %0.5f) [%s]" 
          % (scores.mean(), scores.std(), label))

5-fold cross validation:

Accuracy: 0.7196705970 (+/- 0.01682) [XGB]
Accuracy: 0.7200355605 (+/- 0.01884) [XGB2]
Accuracy: 0.7134541857 (+/- 0.01889) [RF gini]
Accuracy: 0.7134608700 (+/- 0.01850) [StackingClassifier]


In [29]:
clf1.fit(train_X, train_y)
clf2.fit(train_X, train_y)
#clf3.fit(train_X, train_y)
#clf4.fit(train_X, train_y)
sclf.fit(train_X, train_y_numeric)

clf1_pred = clf1.predict(test_X)
clf2_pred = clf2.predict(test_X)
#clf3_pred = clf3.predict(test_X)
#clf4_pred = clf4.predict(test_X)
sclf_pred = pd.Series(sclf.predict(test_X)).replace({1:'F', 0:'N'}).as_matrix()

clf1_acc = accuracy_score(clf1_pred, test_y)
clf2_acc = accuracy_score(clf2_pred, test_y)
#clf3_acc = accuracy_score(clf3_pred, test_y)
#clf4_acc = accuracy_score(clf4_pred, test_y)
sclf_acc = accuracy_score(sclf_pred, test_y)

print('XGB Accuracy: ', clf1_acc)
print('XGB2 Accuracy: ', clf2_acc)
#print('RF Gini Accuracy: ', clf3_acc)
#print('RF entropy: ', clf4_acc)
print('Stacked Classifier Accuracy: ', sclf_acc)

XGB Accuracy:  0.708029197080292
XGB2 Accuracy:  0.7138686131386861
Stacked Classifier Accuracy:  0.710948905109489


In [30]:
print(pd.crosstab(clf1_pred, test_y))
print(pd.crosstab(clf2_pred, test_y))
#print(pd.crosstab(clf3_pred, test_y))
#print(pd.crosstab(clf4_pred, test_y))
print(pd.crosstab(sclf_pred, test_y))

Fatal    F    N
row_0          
F       89   38
N      162  396
Fatal    F    N
row_0          
F       98   43
N      153  391
Fatal    F    N
row_0          
F       84   31
N      167  403


In [31]:
def intersect_fatal_rows(model1, model2, test_X):
    pred1 = model1.predict(test_X)
    pred2 = model2.predict(test_X)
    
    fatal1 = np.where(pred1=='F')
    fatal2 = np.where(pred2=='F')
    
    return(np.intersect1d(fatal1,fatal2))

In [32]:
xgb1 = XGBClassifier(learning_rate=0.01, n_estimators=500, max_depth=4, subsample = 0.8)
xgb1.fit(train_X, train_y)
xgb2 = XGBClassifier(learning_rate=0.1, n_estimators=45, max_depth=3, subsample = 0.8)
xgb2.fit(train_X, train_y)

rF1 = RandomForestClassifier(n_estimators=800, criterion = 'gini', max_depth=10, max_features = 4)
rF1.fit(train_X, train_y)

dummy_test = pd.Series(np.repeat('N', test_y.shape[0]))
dummy_test[intersect_fatal_rows(xgb2, xgb1, test_X)] = 'F'
print('intersected accuracy:', accuracy_score(dummy_test, test_y))
print('not intersected accuracy1 :', accuracy_score(xgb2.predict(test_X), test_y))
print('not intersected accuracy2 :', accuracy_score(xgb1.predict(test_X), test_y))

intersected accuracy: 0.708029197080292
not intersected accuracy1 : 0.7094890510948905
not intersected accuracy2 : 0.708029197080292
